# Pulling and formatting timing files

Once you've finished preprocessing, you'll need to pull the timing files for all of the participants you're interested in running analysis on. This process will pull data from the participants' log files and create time series for each condition of interest. 

The code presented here can be edited (or run) in the terminal by accessing two scripts:
1. B3/scripts/step_3_a_pullingtimings.py
2. B3/scripts/step_3_b_formattingtimings.sh

The first script should pull all of the major conditions for the self-other, peer feedback, and conformity tasks... but it's not yet designed to handle timings for cyberball (which only exports a raw long and will need to be handled differently). I've tried to identify all of the relevant categories of stimuli here, but you will likely want to edit this script as you develop new research questions. 

The second script will adapt all of the timings to the .1D format preferred by AFNI and will stitch together timings from two-run tasks into a single file. 

### Prerequisite steps: 
By the time you get here, you should have: 
* Completed preprocessing for all of the subjects of interest (see the notebooks for STEP_1 and STEP_2)
* Gained access to the scan logs on Box 
* Mapped the DOP-Restricted drive to your computer

#### Note: 
This notebook documents the process of pulling timings that I designed for AFNI analysis. That process works, but it was not designed with the BIDS format in mind. To obtain BIDS-friendly timing files (which may be needed for future analyses that use nipype), refer to BIDS_timings_maker.ipynb (a script that should do this for the first three tasks). 

## Step 1: Downloading and copying the timings

First, you'll need to get the raw log files for the participants you're interested in. Taking care to choose only the logs for the correct scans (check the scan notes), download all of the log files and move them to the proper directory in B3/timings. Log files are formatted like such:
* Self-other run 1 (participant 135): 135_task1_a.csv  -- should go in B3/logs/self_other_all
* Self-other run 2 (participant 135): 135_task1_b.csv -- should go in B3/logs/self_other_all
* Peer feedback run 1 (participant 135): 135_task2_a.csv -- should go in B3/logs/peer_feedback_all
* ...and so on. You get the idea.

For those initial participants who only had one scan, you'll need to rename the log files. Use a similar format as the above so that:
* Self-other (participant 094): 94_task1_socialmedia.csv = 094_task1_ab.csv
* Conformity (participant 094): 94_task3_social-influence.csv = 094_task3_ab.csv

As you might have noticed above, you'll also need to edit some participant IDs to make them three digits long. Just add leading zeros so that 8 = 008, etc. If the file for the correct scan has an extra number at the end (for example task3_a_1.csv), remove that number (to mate it task3_a.csv, for example). 

#### Note: 
This process is tedious, and it would be great if someone could automate it. The only caveat for whoever DOES automate it is to MAKE SURE that the script is not blindly pulling all task1_a and task1_b files for each participant. In many cases, the correct file is task1_a_1.csv or even task1_a_3.csv because of errors during the scan. 

## Step 2: Pulling timings

Next, you'll just run the script B3/scripts/step_three_a_pullingtimings.py. That script contains functions that pull timings for the self-other, peer feedback, and conformity tasks... but nothing yet for Cyberball. Because that script is long and will likely need some tinkering as analysis needs change, I've broken it out section by section in the cells below. 

### Loading the libraries and setting up the button box dictionaries

The first few lines of this script just load in the libraries we'll need (glob, os, numpy, and pandas), then create dictionaries for a few button box responses codes:
* button_keys just assigns the correct number to the color of button pressed in the button box. If things look backwards in analysis, triple check this to make sure it isn't coded backwards (I've checked, but somehow never feel 100% sure due to past analysis traumas). 
* button_keys_inv just reverses the codes, so that the highest rating is coded 1 and the lowest rating is coded 4. 
* button_keys_exp codes the button box responses as if they represent an exponential growth function (each number is e^rating). This is part of a fishing expedition I'm on to explain something about responses peer feedback ratings. The numbers may be changed in the future, because I suspect some modifier other than e needs to be added to flatten the curve a little.  
* button_keys_exp_inv is just the inverse of the growth function... so it represents exponential decay. 

In [13]:
import glob
import numpy as np
import os
import pandas as pd

button_keys = {'b':1,'y':2,'g':3,'r':4} # This encodes the button box responses. THERE IS ALWAYS A CHANCE THIS MIGHT BE BACKWARDS.
button_keys_inv = {'b':4,'y':3,'g':2,'r':1}
button_keys_exp = {'b':2.718, 'y':7.389, 'g':20.086, 'r': 58.594} # This encodes the responeses and transforms them for an exponential relationship. b = e^1, r = e^4
button_keys_exp_inv = {'b': 58.594, 'y': 20.086, 'g': 7.389, 'r': 2.718}

### Pulling the self-other timings

This function pulls all of the self-other timing files from a given .csv. Most of the names here should be obvious to anyone familiar with the task, but here are few descriptions:
* "other_file" is a file containing the timings of all other-generated stimuli
* "self_file" is a file containing the timings of all self-generated simuli
* "like_file" is a file containing the timings of all stimuli the participant rated 3 or 4
* "certain_file" is a file for which the participant gave a certainty rating of 3 or 4
* ... and so on

A few other files of interest are
* "fixation_file" is a file that stores both the timing and the duration of every display of the fixation cross.
* "rating_pmod_file" is a file that stores the timing of each stimuli, paired with the participant's rating of that stimuli.
* "certainty_pmod_file" is a file that stores the timing of each stimuli paired with the participant's certainty rating of that stimuli. This allows us to test whether BOLD signal varies as a linear function of ratings.  
* "rating_pmod_exp_file" is a file that stores the timing of each stimuli paired with e^rating. This allows us to test whether BOLD signal varies as an exponential growth function of ratings. 
* ... and so on.

This function basically just opens a bunch of different files, the uses a bunch of "with" statements to write those files based on the content of certain cells in the .csv. Knowing that, you should be able to build timings for whatever condition you want. Just add a file and a new "with" statement containing the logic. 

You'll notice that all of the timings are the logged moment of stimuli presentation started MINUS "value". This value represents the number of seconds that went by before the scanner was actually started, plus seven seconds to account for the seven seconds we trimmed off the front of each file in preprocessing. The if/elif statement that creates this value is there because the name of that column in our log files changed at some point after scans started.

In [14]:
def selfother_times(file_path, run_id):
    df = pd.read_csv(file_path)
    other_file = f"../timings/{run_id}_selfother_other.txt"
    self_file = f"../timings/{run_id}_selfother_self.txt"
    like_file = f"../timings/{run_id}_selfother_like.txt"
    dislike_file = f"../timings/{run_id}_selfother_dislike.txt"
    self_like_file = f"../timings/{run_id}_selfother_selflike.txt"
    self_dislike_file = f"../timings/{run_id}_selfother_selfdislike.txt"
    other_like_file = f"../timings/{run_id}_selfother_otherlike.txt"
    other_dislike_file = f"../timings/{run_id}_selfother_otherdislike.txt"
    certain_file = f"../timings/{run_id}_selfother_certain.txt"
    uncertain_file = f"../timings/{run_id}_selfother_uncertain.txt"
    fixation_file = f"../timings/{run_id}_selfother_fixation.txt"
    certainty_pmod_file = f"../timings/{run_id}_selfother_certainty_pmod.txt"
    rating_pmod_file = f"../timings/{run_id}_selfother_rating_pmod.txt"
    rating_pmod_inv_file = f"../timings/{run_id}_selfother_rating_pmod_inv.txt"
    rating_pmod_exp_file = f"../timings/{run_id}_selfother_rating_pmod_exp.txt"
    rating_pmod_exp_inv_file = f"../timings/{run_id}_selfother_rating_pmod_exp_inv.txt"
    if 'sm_directions.stopped' in df.columns:
        value = df.loc[1,'sm_directions.started'] + 7
    elif 'sm_instructions.stopped' in df.columns:
        value = df.loc[1,'sm_instructions.started'] + 7
    print(f'reading {file_path}')
    with open(fixation_file, 'w')  as file:
        for index, row in df.iloc[1:].iterrows():
                timing = row['fixation_cross.started']-value
                duration = row['fixation_duration']
                file.write(f'{timing} {duration} \n' )
    with open(other_file, 'w') as file:
        for index, row in df.iloc[1:].iterrows():
            if row['type'] == 'peer':
                timing = row['socialmedia_stim.started']-value
                file.write(f"{timing} \n")
    with open(self_file, 'w') as file:
        for index, row in df.iloc[1:].iterrows():
            if row['type'] == 'self':
                timing = row['socialmedia_stim.started']-value
                file.write(f"{timing} \n")
    with open(like_file, 'w') as file:
        for index, row in df.iloc[1:].iterrows():
            if row['sm_rateresp_box.keys'] == 'r' or row['sm_rateresp_box.keys'] == 'g':
                timing = row['socialmedia_stim.started']-value
                file.write(f"{timing} \n")
    with open(dislike_file, 'w') as file:
        for index, row in df.iloc[1:].iterrows():
            if row['sm_rateresp_box.keys'] == 'b' or row['sm_rateresp_box.keys'] == 'y':
                timing = row['socialmedia_stim.started']-value
                file.write(f"{timing} \n")
    with open(self_like_file, 'w') as file:
        for index, row in df.iloc[1:].iterrows():
            if row['type'] == 'self' and (row['sm_rateresp_box.keys'] == 'r' or row['sm_rateresp_box.keys'] == 'g'):
                timing = row['socialmedia_stim.started']-value
                file.write(f"{timing} \n")
    with open(self_dislike_file, 'w') as file:
        for index, row in df.iloc[1:].iterrows():
            if row['type'] == 'self' and (row['sm_rateresp_box.keys'] == 'b' or row['sm_rateresp_box.keys'] == 'y'):
                timing = row['socialmedia_stim.started']-value
                file.write(f"{timing} \n")
    with open(other_like_file, 'w') as file:
        for index, row in df.iloc[1:].iterrows():
            if row['type'] == 'peer' and (row['sm_rateresp_box.keys'] == 'r' or row['sm_rateresp_box.keys'] == 'g'):
                timing = row['socialmedia_stim.started']-value
                file.write(f"{timing} \n")
    with open(other_dislike_file, 'w') as file:
        for index, row in df.iloc[1:].iterrows():
            if row['type'] == 'peer' and (row['sm_rateresp_box.keys'] == 'b' or row['sm_rateresp_box.keys'] == 'y'):
                timing = row['socialmedia_stim.started']-value
                file.write(f"{timing} \n")
    with open(certain_file, 'w') as file:
        for index, row in df.iloc[1:].iterrows():
           if row['certainty_box.keys'] == 'r' or row['certainty_box.keys'] == 'g':
               timing = row['socialmedia_stim.started']-value
               file.write(f"{timing} \n")
    with open(uncertain_file, 'w') as file:
        for index, row in df.iloc[1:].iterrows():
           if row['certainty_box.keys'] == 'b' or row['certainty_box.keys'] == 'y':
               timing = row['socialmedia_stim.started']-value
               file.write(f"{timing} \n")
    with open(certainty_pmod_file, 'w') as file:
        for index, row in df.iloc[1:].iterrows():
            if row['certainty_box.keys'] in button_keys:
                resp = button_keys.get(row['certainty_box.keys'], None)
                timing = row['socialmedia_stim.started']-value
                file.write(f"{timing}  5  {resp} \n")
    with open(rating_pmod_file, 'w') as file:
        for index, row in df.iloc[1:].iterrows():
            if row['sm_rateresp_box.keys'] in button_keys:
                resp = button_keys.get(row['sm_rateresp_box.keys'], None)
                timing = row['socialmedia_stim.started']-value
                file.write(f"{timing}  5  {resp} \n")
    with open(rating_pmod_inv_file, 'w') as file:
        for index, row in df.iloc[1:].iterrows():
            if row['sm_rateresp_box.keys'] in button_keys:
                resp = button_keys_inv.get(row['sm_rateresp_box.keys'], None)
                timing = row['socialmedia_stim.started']-value
                file.write(f"{timing}  5  {resp} \n")
    with open(rating_pmod_exp_file, 'w') as file:
        for index, row in df.iloc[1:].iterrows():
            if row['sm_rateresp_box.keys'] in button_keys :
                resp = button_keys_exp.get(row['sm_rateresp_box.keys'], None)
                timing = row['socialmedia_stim.started']-value
                file.write(f"{timing}  5  {resp} \n")
    with open(rating_pmod_exp_inv_file, 'w') as file:
        for index, row in df.iloc[1:].iterrows():
            if row['sm_rateresp_box.keys'] in button_keys:
                resp = button_keys_exp_inv.get(row['sm_rateresp_box.keys'], None)
                timing = row['socialmedia_stim.started']-value
                file.write(f"{timing}  5  {resp} \n")


### Pulling the peer feedback timings

This function does the same thing for peer feedback timings. I'll spare you the details of each file, but some important notes are:
* "peer_like" and "peer_dislike" refer to the feedback given by "peers" (peer_like is a rating of 3 or 4)
* "good" and "bad" refer to participant responses to the prompt "How did this feedback make you feel?" (good is 3 or 4)

In [15]:
def peerfeedback_times(file_path, run_id):
    df = pd.read_csv(file_path)
    green_file = f"../timings/{run_id}_peerfeedback_green.txt"
    yellow_file = f"../timings/{run_id}_peerfeedback_yellow.txt"
    orange_file = f"../timings/{run_id}_peerfeedback_orange.txt"
    peer_like_file = f"../timings/{run_id}_peerfeedback_peer_like.txt"
    peer_dislike_file = f"../timings/{run_id}_peerfeedback_peer_dislike.txt"
    good_file = f"../timings/{run_id}_peerfeedback_good.txt"
    bad_file = f"../timings/{run_id}_peerfeedback_bad.txt"
    green_like_file = f"../timings/{run_id}_peerfeedback_green_like.txt"
    green_dislike_file = f"../timings/{run_id}_peerfeedback_green_dislike.txt"
    yellow_like_file = f"../timings/{run_id}_peerfeedback_yellow_like.txt"
    yellow_dislike_file = f"../timings/{run_id}_peerfeedback_yellow_dislike.txt"
    orange_like_file = f"../timings/{run_id}_peerfeedback_orange_like.txt"
    orange_dislike_file = f"../timings/{run_id}_peerfeedback_orange_dislike.txt"
    fixation_file = f"../timings/{run_id}_peerfeedback_fixation.txt"
    feedback_pmod_file = f"../timings/{run_id}_peerfeedback_feedback_pmod.txt"
    feeling_pmod_file = f"../timings/{run_id}_peerfeedback_feeling_pmod.txt"
    feedback_pmod_inv_file = f"../timings/{run_id}_peerfeedback_feedback_pmod_inv.txt"
    feeling_pmod_inv_file = f"../timings/{run_id}_peerfeedback_feeling_pmod_inv.txt"
    feedback_pmod_exp_file = f"../timings/{run_id}_peerfeedback_feedback_pmod_exp.txt"
    feeling_pmod_exp_file = f"../timings/{run_id}_peerfeedback_feeling_pmod_exp.txt"
    feedback_pmod_exp_inv_file = f"../timings/{run_id}_peerfeedback_feedback_pmod_exp_inv.txt"
    feeling_pmod_exp_inv_file = f"../timings/{run_id}_peerfeedback_feeling_pmod_exp_inv.txt"
    value = df.loc[1,'peerfeedback_directions.started'] + 7
    print(f'reading {file_path}')
    df['peer_stim_rating'] = pd.to_numeric(df['peer_stim_rating'].fillna(0).astype(int))  
    with open(fixation_file, 'w')  as file:
        for index, row in df.iloc[1:].iterrows():
                timing = row['fixation_cross.started']-value
                duration = row['fixation_duration']
                file.write(f'{timing} {duration} \n' )
    with open(green_file, 'w') as file:
        for index, row in df.iloc[1:].iterrows():
            if row['border_color'] == 'green':
                timing = row['peerfeedback_stimuli.started']-value
                file.write(f"{timing} \n")
    with open(green_file, 'w') as file:
        for index, row in df.iloc[1:].iterrows():
            if row['border_color'] == 'green':
                timing = row['peerfeedback_stimuli.started']-value
                file.write(f"{timing} \n")
    with open(yellow_file, 'w') as file:
        for index, row in df.iloc[1:].iterrows():
            if row['border_color'] == 'yellow':
                timing = row['peerfeedback_stimuli.started']-value
                file.write(f"{timing} \n")
    with open(orange_file, 'w') as file:
        for index, row in df.iloc[1:].iterrows():
            if row['border_color'] == 'orange':
                timing = row['peerfeedback_stimuli.started']-value
                file.write(f"{timing} \n")
    with open(peer_like_file, 'w') as file:
        for index, row in df.iloc[1:].iterrows():
            if row['peer_stim_rating'] > 2:
                timing = row['peerfeedback_stimuli.started']-value
                file.write(f"{timing} \n")
    with open(peer_dislike_file, 'w') as file:
        for index, row in df.iloc[1:].iterrows():
            if row['peer_stim_rating'] < 3:
                timing = row['peerfeedback_stimuli.started']-value
                file.write(f"{timing} \n")
    with open(good_file, 'w') as file:
        for index, row in df.iloc[1:].iterrows():
            if row['feeling_rep.keys'] == 'g' or row['feeling_rep.keys'] == 'r':
                timing = row['peerfeedback_stimuli.started']-value
                file.write(f"{timing} \n")
    with open(bad_file, 'w') as file:
        for index, row in df.iloc[1:].iterrows():
            if row['feeling_rep.keys'] == 'b' or row['feeling_rep.keys'] == 'y':
                timing = row['peerfeedback_stimuli.started']-value
                file.write(f"{timing} \n")
    with open(green_like_file, 'w') as file:
        for index, row in df.iloc[1:].iterrows():
            if row['border_color'] == 'green' and row['peer_stim_rating'] > 2:
                timing = row['peerfeedback_stimuli.started']-value
                file.write(f"{timing} \n")
    with open(yellow_like_file, 'w') as file:
        for index, row in df.iloc[1:].iterrows():
            if row['border_color'] == 'yellow' and row['peer_stim_rating'] > 2:
                timing = row['peerfeedback_stimuli.started']-value
                file.write(f"{timing} \n")
    with open(orange_like_file, 'w') as file:
        for index, row in df.iloc[1:].iterrows():
            if row['border_color'] == 'orange' and row['peer_stim_rating'] > 2:
                timing = row['peerfeedback_stimuli.started']-value
                file.write(f"{timing} \n")
    with open(green_dislike_file, 'w') as file:
        for index, row in df.iloc[1:].iterrows():
            if row['border_color'] == 'green' and row['peer_stim_rating'] < 3:
                timing = row['peerfeedback_stimuli.started']-value
                file.write(f"{timing} \n")
    with open(yellow_dislike_file, 'w') as file:
        for index, row in df.iloc[1:].iterrows():
            if row['border_color'] == 'yellow' and row['peer_stim_rating'] < 3:
                timing = row['peerfeedback_stimuli.started']-value
                file.write(f"{timing} \n")
    with open(orange_dislike_file, 'w') as file:
        for index, row in df.iloc[1:].iterrows():
            if row['border_color'] == 'orange' and row['peer_stim_rating'] < 3:
                timing = row['peerfeedback_stimuli.started']-value
                file.write(f"{timing} \n")
    with open(feedback_pmod_file, 'w') as file:
        for index, row in df.iloc[1:].iterrows():
            if row['feeling_rep.keys'] in button_keys:
                resp = row['peer_stim_rating']
                timing = row['peerfeedback_stimuli.started']-value
                file.write(f"{timing}  8  {resp} \n")
    with open(feeling_pmod_file, 'w') as file:
        for index, row in df.iloc[1:].iterrows():
            if row['feeling_rep.keys'] in button_keys:
                resp = button_keys.get(row['feeling_rep.keys'], None)
                timing = row['peerfeedback_stimuli.started']-value
                file.write(f"{timing}  8  {resp} \n")
    with open(feedback_pmod_inv_file, 'w') as file:
        for index, row in df.iloc[1:].iterrows():
            if row['feeling_rep.keys'] in button_keys:
                resp = 5 - row['peer_stim_rating']
                timing = row['peerfeedback_stimuli.started']-value
                file.write(f"{timing}  8  {resp} \n")
    with open(feeling_pmod_inv_file, 'w') as file:
        for index, row in df.iloc[1:].iterrows():
            if row['feeling_rep.keys'] in button_keys:
                resp = button_keys_inv.get(row['feeling_rep.keys'], None)
                timing = row['peerfeedback_stimuli.started']-value
                file.write(f"{timing}  8  {resp} \n")
    with open(feedback_pmod_exp_file, 'w') as file:
        for index, row in df.iloc[1:].iterrows():
            if row['feeling_rep.keys'] in button_keys:
                resp = np.exp(row['peer_stim_rating'])
                timing = row['peerfeedback_stimuli.started']-value
                file.write(f"{timing}  8  {resp} \n")
    with open(feeling_pmod_exp_file, 'w') as file:
        for index, row in df.iloc[1:].iterrows():
            if row['feeling_rep.keys'] in button_keys:
                resp = button_keys_exp.get(row['feeling_rep.keys'], None)
                timing = row['peerfeedback_stimuli.started']-value
                file.write(f"{timing}  8  {resp} \n")
    with open(feedback_pmod_exp_inv_file, 'w') as file:
        for index, row in df.iloc[1:].iterrows():
            if row['feeling_rep.keys'] in button_keys:
                resp = np.exp(5 - row['peer_stim_rating'])
                timing = row['peerfeedback_stimuli.started']-value
                file.write(f"{timing}  8  {resp} \n")
    with open(feeling_pmod_exp_inv_file, 'w') as file:
        for index, row in df.iloc[1:].iterrows():
            if row['feeling_rep.keys'] in button_keys:
                resp = button_keys_exp_inv.get(row['feeling_rep.keys'], None)
                timing = row['peerfeedback_stimuli.started']-value
                file.write(f"{timing}  8  {resp} \n")

### Pulling the conformity timings

This functions pulls timings from the conformity task. It's basically the same as the other two functions, but also has a bit of code at the beginning that pulls the "certainty" ratings from the self-other task. So for this code to work, you'll need to have also uploaded the logs for that participant's self-other task. 

#### Notes:
This script notes changes in ratings after feedback, with SHOULD but might not always equate to conformity. In the future, it might be a good idea to parse out change that aligned with participant feedback from changes that went *against* participant feedback (for example, when a participant learns that peer ratings are LOWER than theirs and in turn makes their rating *higher* than the original).  

Right now this function only pulls timings for the presentation of peer feedback (3 seconds). Depending on the research questions, it might also be important to pull timings for other parts of the process (such as when participants are reminded of their own ratings). 


In [16]:
def conformity_times(file_path, run_id, prefix, suffix):
    df = pd.read_csv(file_path)
    # This section of code pulls the Task 1 files and adds a column for certainty
    selfother_search_pattern = f'../logs/selfother_all/{prefix}_task1*.csv'
    selfother_logs = glob.glob(selfother_search_pattern)
    selfother_dfs = [pd.read_csv(file) for file in selfother_logs]
    if len(selfother_dfs) > 1:
         selfother_merged = pd.concat(selfother_dfs, ignore_index = True)
         df = df.merge(selfother_merged[['stimuli_file','certainty_box.keys']], on = 'stimuli_file', how = 'left')
         df = df.iloc[1:].reset_index(drop=True)
    else:
         df = df.merge(selfother_dfs[0][['stimuli_file', 'certainty_box.keys']], on='stimuli_file', how='left')
    # Now back to our regular programming (lol)
    same_file = f"../timings/{run_id}_conformity_same.txt"
    lower_file = f"../timings/{run_id}_conformity_lower.txt"
    higher_file = f"../timings/{run_id}_conformity_higher.txt"
    misaligned_file = f"../timings/{run_id}_conformity_misaligned.txt"
    change_file = f"../timings/{run_id}_conformity_change.txt"
    nochange_file = f"../timings/{run_id}_conformity_nochange.txt"
    same_change_file = f"../timings/{run_id}_conformity_same_change.txt"
    same_nochange_file = f"../timings/{run_id}_conformity_same_nochange.txt"
    lower_change_file = f"../timings/{run_id}_conformity_lower_change.txt"
    lower_nochange_file = f"../timings/{run_id}_conformity_lower_nochange.txt"
    higher_change_file = f"../timings/{run_id}_conformity_higher_change.txt"
    higher_nochange_file = f"../timings/{run_id}_conformity_higher_nochange.txt"
    like_change_file = f"../timings/{run_id}_conformity_like_change.txt"
    like_nochange_file = f"../timings/{run_id}_conformity_like_nochange.txt"
    dislike_change_file = f"../timings/{run_id}_conformity_dislike_change.txt"
    dislike_nochange_file = f"../timings/{run_id}_conformity_dislike_nochange.txt"
    certain_change_file = f"../timings/{run_id}_conformity_certain_change.txt"
    certain_nochange_file = f"../timings/{run_id}_conformity_certain_nochange.txt"
    uncertain_change_file = f"../timings/{run_id}_conformity_uncertain_change.txt"
    uncertain_nochange_file = f"../timings/{run_id}_conformity_uncertain_nochange.txt"
    fixation_file = f"../timings/{run_id}_conformity_fixation.txt"
    value = df.loc[1,'si_directions.started'] + 7
    print(f'reading {file_path}')
    with open(fixation_file, 'w') as file:
        for index, row in df.iloc[1:].iterrows():
                timing = row['fixation_cross.started']-value
                duration = row['fixation_duration']
                file.write(f'{timing} {duration} \n' )
    with open(same_file, 'w') as file:
        for index, row in df.iloc[1:].iterrows():
            if row['peer_rating'] == 'SAME':
                timing = row['peer_rate.started']-value
                file.write(f"{timing} \n")
    with open(lower_file, 'w') as file:
        for index, row in df.iloc[1:].iterrows():
            if row['peer_rating'] == 'LOWER':
                timing = row['peer_rate.started']-value
                file.write(f"{timing} \n")
    with open(higher_file, 'w') as file:
        for index, row in df.iloc[1:].iterrows():
            if row['peer_rating'] == 'LOWER':
                timing = row['peer_rate.started']-value
                file.write(f"{timing} \n")
    with open(misaligned_file, 'w') as file:
        for index, row in df.iloc[1:].iterrows():
            if row['sm_resp'] != 'None' and (row['peer_rating'] == 'LOWER' or row['peer_rating'] == 'HIGHER'):
                timing = row['peer_rate.started']-value
                file.write(f"{timing} \n")
    with open(change_file, 'w') as file:
        for index, row in df.iloc[1:].iterrows():
            if row['sm_resp'] != 'None' and row['sm_resp'] != row['rerate_box.keys']:
                timing = row['peer_rate.started']-value
                file.write(f"{timing} \n")
    with open(nochange_file, 'w') as file:
        for index, row in df.iloc[1:].iterrows():
            if row['sm_resp'] != 'None' and row['sm_resp'] == row['rerate_box.keys']:
                timing = row['peer_rate.started']-value
                file.write(f"{timing} \n")
    with open(same_change_file, 'w') as file:
        for index, row in df.iloc[1:].iterrows():
            if row['sm_resp'] != 'None' and row['sm_resp'] != row['rerate_box.keys'] and row['peer_rating'] == 'SAME':
                timing = row['peer_rate.started']-value
                file.write(f"{timing} \n")
    with open(same_nochange_file, 'w') as file:
        for index, row in df.iloc[1:].iterrows():
            if row['sm_resp'] != 'None' and row['sm_resp'] == row['rerate_box.keys'] and row['peer_rating'] == 'SAME':
                timing = row['peer_rate.started']-value
                file.write(f"{timing} \n")
    with open(lower_change_file, 'w') as file:
        for index, row in df.iloc[1:].iterrows():
            if row['sm_resp'] != 'None' and row['sm_resp'] != row['rerate_box.keys'] and row['peer_rating'] == 'LOWER':
                timing = row['peer_rate.started']-value
                file.write(f"{timing} \n")
    with open(lower_nochange_file, 'w') as file:
        for index, row in df.iloc[1:].iterrows():
            if row['sm_resp'] != 'None' and row['sm_resp'] == row['rerate_box.keys'] and row['peer_rating'] == 'LOWER':
                timing = row['peer_rate.started']-value
                file.write(f"{timing} \n")
    with open(higher_change_file, 'w') as file:
        for index, row in df.iloc[1:].iterrows():
            if row['sm_resp'] != 'None' and row['sm_resp'] != row['rerate_box.keys'] and row['peer_rating'] == 'HIGHER':
                timing = row['peer_rate.started']-value
                file.write(f"{timing} \n")
    with open(higher_nochange_file, 'w') as file:
        for index, row in df.iloc[1:].iterrows():
            if row['sm_resp'] != 'None' and row['sm_resp'] == row['rerate_box.keys'] and row['peer_rating'] == 'HIGHER':
                timing = row['peer_rate.started']-value
                file.write(f"{timing} \n")
    with open(like_change_file, 'w') as file:
        for index, row in df.iloc[1:].iterrows():
            if row['sm_resp'] in button_keys:
                resp = button_keys_inv.get(row['sm_resp'], None)
                if resp > 2 and row['sm_resp'] == row['rerate_box.keys']:
                   timing = row['peer_rate.started']-value
                   file.write(f"{timing} \n")
    with open(like_nochange_file, 'w') as file:
        for index, row in df.iloc[1:].iterrows():
            if row['sm_resp'] in button_keys:
                resp = button_keys_inv.get(row['sm_resp'], None)
                if resp > 2 and row['sm_resp'] != row['rerate_box.keys']:
                   timing = row['peer_rate.started']-value
                   file.write(f"{timing} \n")
    with open(dislike_change_file, 'w') as file:
        for index, row in df.iloc[1:].iterrows():
            if row['sm_resp'] in button_keys:
                resp = button_keys_inv.get(row['sm_resp'], None)
                if resp < 3 and row['sm_resp'] == row['rerate_box.keys']:
                   timing = row['peer_rate.started']-value
                   file.write(f"{timing} \n")
    with open(dislike_nochange_file, 'w') as file:
        for index, row in df.iloc[1:].iterrows():
            if row['sm_resp'] in button_keys:
                resp = button_keys_inv.get(row['sm_resp'], None)
                if resp < 3 and row['sm_resp'] != row['rerate_box.keys']:
                   timing = row['peer_rate.started']-value
                   file.write(f"{timing} \n")
    with open(certain_change_file, 'w') as file:
        for index, row in df.iloc[1:].iterrows():
            if row['sm_resp'] in button_keys and row['certainty_box.keys'] in button_keys:
                resp = button_keys_inv.get(row['certainty_box.keys'], None)
                if resp > 2 and row['sm_resp'] == row['rerate_box.keys']:
                   timing = row['peer_rate.started']-value
                   file.write(f"{timing} \n")
    with open(certain_nochange_file, 'w') as file:
        for index, row in df.iloc[1:].iterrows():
            if row['sm_resp'] in button_keys and row['certainty_box.keys'] in button_keys:
                resp = button_keys_inv.get(row['certainty_box.keys'], None)
                if resp > 2 and row['sm_resp'] != row['rerate_box.keys']:
                   timing = row['peer_rate.started']-value
                   file.write(f"{timing} \n")
    with open(uncertain_change_file, 'w') as file:
        for index, row in df.iloc[1:].iterrows():
            if row['sm_resp'] in button_keys and row['certainty_box.keys'] in button_keys:
                resp = button_keys_inv.get(row['certainty_box.keys'], None)
                if resp < 3 and row['sm_resp'] == row['rerate_box.keys']:
                   timing = row['peer_rate.started']-value
                   file.write(f"{timing} \n")
    with open(uncertain_nochange_file, 'w') as file:
        for index, row in df.iloc[1:].iterrows():
            if row['sm_resp'] in button_keys and row['certainty_box.keys'] in button_keys:
                resp = button_keys_inv.get(row['certainty_box.keys'], None)
                if resp < 3 and row['sm_resp'] != row['rerate_box.keys']:
                   timing = row['peer_rate.started']-value
                   file.write(f"{timing} \n")


### Pulling the Cyberball timings. 

This function hasn't been built yet, so if you try to pull these timings it will nag you to write the code.

#### Note: 
The tough thing about CyberBall is that it doesn't export a .csv log like the other tasks, but instead just a raw .txt log. I've held off on coding a timing puller for it because I suspect that someone in one of our Co-I's labs has a tool for this already. 

In [17]:
def cyberball_times(file_path, run_id):
    print('Please code this function :)')

### The one that binds them 

This function is pretty simple. Depending on the task you choose (the script will ask you to type it in), it will run the appropriate function over all of the .csvs in the appropriate log directory. We could just have this script do everything for all of the tasks at once, but I guess I was having fun with functions when I coded this.

In [21]:
def process_all_csvs(directory,task):
    for filename in os.listdir(directory):
        if filename.endswith('.csv'):
            parts = filename.split('_')
            prefix = parts[0]
            suffix = parts[-1].replace('.csv','')
            file_path = os.path.join(directory, filename)
            run_id = f"{prefix}{suffix}"
            if task == 'self-other':
               selfother_times(file_path, run_id)
            if task == 'feedback':
               peerfeedback_times(file_path, run_id)
            if task == 'conformity':
               conformity_times(file_path, run_id, prefix,suffix)
            if task == 'cyberball':
               cyberball_times(file_path, run_id)

### The interactive part

This last part of the script just serves up an input on the screen. It'll ask you what task you want timings for, and then you'll type in the correct answer. Don't add a space or anything to the end of your answer. 

#### Note:
If you end up changing the structure or location of the logs for any reason, you'll need to change this code to reflect that. 

In [22]:
task = input('What task would you like timings for? (self-other, feedback, conform, cyberball) ')

if task == 'self-other':
    directory = '../logs/selfother_all'
if task == 'feedback':
    directory = '../logs/peerfeedback_all'
if task == 'conform':
    directory = '../logs/conformity_all'
if task == 'cyberball':
    directory = '../logs/cyberball'

process_all_csvs(directory,task)

What task would you like timings for? (self-other, feedback, conform, cyberball) feedback
reading ../logs/peerfeedback_all/008_task2_a.csv
reading ../logs/peerfeedback_all/008_task2_b.csv
reading ../logs/peerfeedback_all/038_task2_a.csv
reading ../logs/peerfeedback_all/038_task2_b.csv
reading ../logs/peerfeedback_all/055_task2_a.csv
reading ../logs/peerfeedback_all/055_task2_b.csv
reading ../logs/peerfeedback_all/058_task2_a.csv
reading ../logs/peerfeedback_all/058_task2_b.csv
reading ../logs/peerfeedback_all/079_task2_a.csv
reading ../logs/peerfeedback_all/079_task2_b.csv
reading ../logs/peerfeedback_all/094_task2_ab.csv
reading ../logs/peerfeedback_all/104_task2_ab.csv
reading ../logs/peerfeedback_all/110_task2_a.csv
reading ../logs/peerfeedback_all/110_task2_b.csv
reading ../logs/peerfeedback_all/135_task2_a.csv
reading ../logs/peerfeedback_all/135_task2_b.csv
reading ../logs/peerfeedback_all/142_task2_ab.csv
reading ../logs/peerfeedback_all/143_task2_a.csv
reading ../logs/peerfeedb

## Step 3: Formatting the timings

All of this work has gotten us a whole lot of .txt files, which contain lists of timings for each condition (and in the case of pmod timings, also the duration and hypothesized amplitude modulation). Before we can use these in a regression, though, we need to convert them to the .1D format preferred by AFNI. This script uses an AFNI python tool to:
1. Merge timing files together into one (if needed)
2. Shift the timing files from long-format .txt files to wide-format .1D files
3. "Marry" the modulation data with the timing if it exists
4. Save the new files in a specific directory depending on whether they are one-run or two-run files. 

This script is long, repetitive, and changes often. For those reasons, I've just included a really short snippet that should give you some idea of what's going on. To see it all, and to run it, check out B3/scripts/step_three_b_formattingtimings.sh. 

A few notes about it:
* Right now it's designed to format ALL of the files that can be pulled for ALL tasks. If you only want to do this for one specific task, you'll need to comment out all of the other lines by adding "#" to the start of those lines. 


* If you want to create new files in the previous step, you'll need to add lines for them here. Remember to add both two-run (the first "if" loop) and one-run (the second "if" loop) options.


* Right now this pulls from all subjects listed in the file subjList.txt. To change the subjects, either edit this file (nano subjList.txt) or just replace "\`cat subjList.txt\`" with the subject IDs. (Note that this includes removing the \` symbols).


* The "file1" and "file2" at the top are used to determine if the subject has one or two runs. If a subject has this particular file coded for one or two runs, it will treat ALL files for the subject that way. Thus, any participant who has two runs of MOST tasks but one run for a single task (due to an error during the scan) will need some special treatment. If you run this script in the terminal, you'll see an error for that participant. I recommend building a custom script for them or just running the correct commands directly in the command line. 


* For each participant, this script will tell you the participant is NOT a one-run or two-run participant. If the script says BOTH of these things for any given participant, there is something wrong with their files.

#### Note:
This is DEFINITELY a way to make this code far shorter and more efficient by calling in the filenames from an external list (just as we can do for subjects). But I haven't done that yet --- if someone DOES decide to do that, just be mindful of the fact that some of these timings have one file while others have two. 

In [12]:
%%bash 

for subj in `cat subjList.txt`; do

file1 = "../timings/${subj}a_peerfeedback_bad.txt"
file2 = "../timings/${subj}ab_peerfeedback_bad.txt"

if [ -e "${file1}" ]; then

  timing_tool.py -fsl_timing_files "../timings/${subj}a_selfother_other.txt" "../timings/${subj}b_selfother_other.txt" -write_timing "../timings/tworun1D/${subj}.selfother_other.1D"
  timing_tool.py -fsl_timing_files "../timings/${subj}a_peerfeedback_bad.txt" "../timings/${subj}b_peerfeedback_bad.txt" -write_timing "../timings/tworun1D/${subj}.peerfeedback_bad.1D"
  timing_tool.py -fsl_timing_files "../timings/${subj}a_conformity_same.txt" "../timings/${subj}b_conformity_same.txt" -write_timing "../timings/tworun1D/${subj}.conformity_same.1D"

else
echo ${subj} " is not a two-run participant"

fi

if [ -e "${file2}" ]; then

  timing_tool.py -fsl_timing_files "../timings/${subj}ab_selfother_other.txt" -write_timing "../timings/onerun1D/${subj}.selfother_other.1D"
  timing_tool.py -fsl_timing_files "../timings/${subj}ab_peerfeedback_bad.txt" -write_timing "../timings/onerun1D/${subj}.peerfeedback_bad.1D"
  timing_tool.py -fsl_timing_files "../timings/${subj}ab_conformity_same.txt" -write_timing "../timings/onerun1D/${subj}.conformity_same.1D"

else 
echo ${subj} " is not a one-run participant"

fi 

done

008  is not a two-run participant
008  is not a one-run participant
038  is not a two-run participant
038  is not a one-run participant
055  is not a two-run participant
055  is not a one-run participant
058  is not a two-run participant
058  is not a one-run participant
079  is not a two-run participant
079  is not a one-run participant
094  is not a two-run participant
094  is not a one-run participant
104  is not a two-run participant
104  is not a one-run participant
110  is not a two-run participant
110  is not a one-run participant
127  is not a two-run participant
127  is not a one-run participant
135  is not a two-run participant
135  is not a one-run participant
142  is not a two-run participant
142  is not a one-run participant
143  is not a two-run participant
143  is not a one-run participant
145  is not a two-run participant
145  is not a one-run participant
148  is not a two-run participant
148  is not a one-run participant
177  is not a two-run participant
177  is not a 

bash: line 4: file1: command not found
bash: line 5: file2: command not found
bash: line 4: file1: command not found
bash: line 5: file2: command not found
bash: line 4: file1: command not found
bash: line 5: file2: command not found
bash: line 4: file1: command not found
bash: line 5: file2: command not found
bash: line 4: file1: command not found
bash: line 5: file2: command not found
bash: line 4: file1: command not found
bash: line 5: file2: command not found
bash: line 4: file1: command not found
bash: line 5: file2: command not found
bash: line 4: file1: command not found
bash: line 5: file2: command not found
bash: line 4: file1: command not found
bash: line 5: file2: command not found
bash: line 4: file1: command not found
bash: line 5: file2: command not found
bash: line 4: file1: command not found
bash: line 5: file2: command not found
bash: line 4: file1: command not found
bash: line 5: file2: command not found
bash: line 4: file1: command not found
bash: line 5: file2: comm

## That's it! 

Congrats! You've just pulled all of the timing files you'll need for your analysis. As a next step, check out the STEP_4_First_Level_Analysis notebook in the directory. 

It's also generally a good practice to actually look at a few of the .1D timing files you've produced to make sure they align with the timings you'd expect given that participant's logs. Remember that all of the numbers will be smaller than the timing numbers in the logs by a value that varies across runs (but is the same for every timing in each single run).


Thanks and cheers,

Matt